
# Amazon Reviews Scraping

In [10]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [18]:
# Header to set the requests as a browser requests
'''
These headers are used to mimic a browser request. This is a common 
practice in web scraping to prevent websites from blocking your requests.
'''
headers = {
   'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
     'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [19]:
# url of The amazon Review page
reviews_url = 'https://www.amazon.com/Legendary-Whitetails-Journeyman-Jacket-Tarmac/product-reviews/B013KW38RQ/'

In [20]:
# number of pages to scrape.
len_page = 6


### Functions

In [21]:
# This function retrieves the HTML data from multiple pages of 
# Amazon reviews.

# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [22]:
# This function extracts relevant information from the HTML data of each review.
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

### Data Process

In [23]:

html_datas = reviewsHtml(reviews_url, len_page)

ConnectionError: HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Legendary-Whitetails-Journeyman-Jacket-Tarmac/product-reviews/B013KW38RQ/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BDB7079190>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [24]:
# Empty List to Hold all reviews data
reviews = []

In [9]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

NameError: name 'html_datas' is not defined

In [41]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [42]:
df_reviews

,Name,Stars,Title,Date,Description
0,Nick,5.0,"5.0 out of 5 stars\nBought twice, would buy again",24/02/2024,I bought this jacket first five years ago and ...
1,Bartek,5.0,5.0 out of 5 stars\nRugged Comfort Meets Style...,19/03/2024,The Legendary Whitetails Journeyman Shirt Jack...
2,E. McDaniel,5.0,5.0 out of 5 stars\nA++ Quality and Fit; my fa...,19/02/2024,My husband loves everything about this casual ...
3,Jeff,4.0,"4.0 out of 5 stars\nRugged, good look and nice...",02/10/2018,The short answer to if you should go down from...
4,ROB P.,5.0,5.0 out of 5 stars\nCool-lookin Shacket,30/03/2024,"I purchased the Tobacco color, and thrilled wi..."
5,David S.,5.0,5.0 out of 5 stars\nNice jacket - slim fit,07/04/2024,I have a slim build and the medium fit me perf...
6,M. Weber,3.0,"3.0 out of 5 stars\nNeck is massive, great fin...",09/03/2024,The fabric is really quite nice. Silky and dec...
7,Amazon Customer,5.0,5.0 out of 5 stars\nFits and looks great,13/04/2024,I really like the legendary style and quality ...
8,Matt,4.0,4.0 out of 5 stars\nNice jacket - could be better,23/03/2024,I like the jacket a lot and will probably wear...
9,Marty Harbison,5.0,5.0 out of 5 stars\nVery well made!,16/04/2024,Wanted a lightweight warm jacket that was not ...


In [43]:
df_reviews.size

200

In [44]:
df_reviews.shape

(40, 5)

In [45]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)

In [46]:
df_reviews.to_html('reviews.html', index=False)